In [1]:
import SimpleITK as sitk
import numpy as np
import json
import cv2
import copy
from tqdm import tqdm
from tqdm.contrib import tzip
import os
from glob import glob
import random,json

In [12]:
if os.path.exists('hospital_data_clean') == False:
    
    os.mkdir('hospital_data_clean')
    os.mkdir('hospital_data_clean/png/')
    os.mkdir('hospital_data_clean/dcm/')
    os.mkdir('hospital_data_clean/png/origin')
    os.mkdir('hospital_data_clean/png/mask')
    os.mkdir('hospital_data_clean/dcm/origin')
    os.mkdir('hospital_data_clean/dcm/mask')

In [13]:
nii_path_list = glob("hospital_data/*.nii.gz")
# nii_path_list = [i for i in nii_path_list if 'vascular' not in i ]
nii_path_list.sort()

In [14]:
for nii_path in tqdm(nii_path_list):

    reader = sitk.ImageFileReader()
    reader.SetFileName(nii_path)
    image = reader.Execute()
    img_array = sitk.GetArrayFromImage(image)
    
    if 'vascular' not in nii_path:
        volumn_name = nii_path.rsplit('/',1)[1].split('.')[0] 
        out_path1 = 'hospital_data_clean/png/origin'
        out_path2 = 'hospital_data_clean/dcm/origin'
        
        for num in range(img_array.shape[0]):
            img_array1 = img_array[num,:,:]
            cv2.imwrite(os.path.join(out_path1,'{0}_{1}.png'.format(volumn_name,num)),img_array1)
            
            img_out_itk = sitk.GetImageFromArray(img_array1.astype('int16'))
            sitk.WriteImage(img_out_itk, os.path.join(out_path2 , '{0}_{1}.dcm'.format(volumn_name,num)))
    
    else:
        volumn_name = nii_path.rsplit('/',1)[1].split('_')[0] 
        out_path1 = 'hospital_data_clean/png/mask'
        out_path2 = 'hospital_data_clean/dcm/mask'
        
        for num in range(img_array.shape[0]):
            img_array1 = img_array[num,:,:]
            cv2.imwrite(os.path.join(out_path1,'{0}_{1}.png'.format(volumn_name,num)),img_array1)
            
            img_out_itk = sitk.GetImageFromArray(img_array1.astype('int16'))
            sitk.WriteImage(img_out_itk,  os.path.join(out_path2 , '{0}_{1}.dcm'.format(volumn_name,num)))

100%|██████████| 40/40 [06:05<00:00,  9.13s/it]


In [2]:
nii_path_list = glob("hospital_data/*rebi.nii.gz")
# nii_path_list = [i for i in nii_path_list if 'vascular' not in i ]
nii_path_list.sort()

for nii_path in tqdm(nii_path_list):

    reader = sitk.ImageFileReader()
    reader.SetFileName(nii_path)
    image = reader.Execute()
    img_array = sitk.GetArrayFromImage(image)  

    volumn_name = nii_path.rsplit('/',1)[1].split('_')[0] 
    out_path1 = 'hospital_data_clean/png/rebi_mask'
    out_path2 = 'hospital_data_clean/dcm/rebi_mask'
    try:
        os.mkdir(out_path1)
        os.mkdir(out_path2)
    except:
        None
    for num in range(img_array.shape[0]):
        img_array1 = img_array[num,:,:]
        cv2.imwrite(os.path.join(out_path1,'{0}_{1}.png'.format(volumn_name,num)),img_array1)
        
        img_out_itk = sitk.GetImageFromArray(img_array1.astype('int16'))
        sitk.WriteImage(img_out_itk,  os.path.join(out_path2 , '{0}_{1}.dcm'.format(volumn_name,num)))

100%|██████████| 20/20 [00:33<00:00,  1.68s/it]


In [ ]:
# 取出1/10作为训练集，另外9/10作为测试集,做成json的格式记录,构造finetune数据集
name = 'origin'

name2 = 'mask_bone'

allpic = glob('hospital_data_clean/png/{0}/*.png'.format(name))

sample_num = int(len(allpic)/10)
train_pic = random.sample(allpic, sample_num)
test_pic = list(set(allpic) - set(train_pic))
if name == 'origin':
    with open('hospital_data_clean/png/finetune/train.json','w') as f:
        json.dump(train_pic, f, indent=4)
    
    with open('hospital_data_clean/png/finetune/test.json','w') as f:
        json.dump(test_pic, f, indent=4)

# copy
import shutil

for i in tqdm(train_pic):
    shutil.copy(i,'hospital_data_clean/png/finetune/train/{0}/'.format(name))
    shutil.copy(i.replace(name, name2),'hospital_data_clean/png/finetune/train/mask/')

for j in tqdm(test_pic):
    shutil.copy(j,'hospital_data_clean/png/finetune/test/{0}/'.format(name))
    shutil.copy(j.replace(name, name2),'hospital_data_clean/png/finetune/test/mask/')

In [15]:
from check_all import check_all
check_all('hospital_data_clean/png/mask','hospital_data_clean/png/origin',(512,512,3))

  0%|          | 0/7007 [00:00<?, ?it/s]

There're nothing wrong


In [ ]:
from check_all import check_all
check_all('hospital_data_clean/dcm/mask','hospital_data_clean/dcm/origin',(512,512,3))